In [ ]:
import numpy as np
import pandas as pd
from functools import partial
from importlib import reload
from collections import Counter
import copy
import matplotlib.pyplot as plt

from TunaSims import tuna_sim
import funcOb
import math_distance
import tools

import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

Manhattan Similarity Demo

In [ ]:
demo_matches = pd.read_pickle('/Users/jonahpoczobutt/projects/TunaRes/metlinGnps_NIST20_matchedPol/intermediateOutputs/splitMatches/train/10_ppm/chunk_1.pkl')
demo_query = demo_matches.iloc[0]['query']
demo_target = demo_matches.iloc[0]['target']
demo_query_prec = demo_matches.iloc[0]['precquery']
demo_target_prec = demo_matches.iloc[0]['prectarget']

In [ ]:
manhattan_tuna = tuna_sim(demo_query,
              demo_target,
              demo_query_prec,
              demo_target_prec,
              query_da_thresh = 0.0,
              target_da_thresh = 0.0,
              dif_a = 1,
              dif_b = 1,
              unnormed=1)

demo_query[:,1] /= sum(demo_query[:,1])
demo_target[:,1] /= sum(demo_target[:,1])
combined_old = tools.match_peaks_in_spectra(demo_query, demo_target, ms2_da=0.05)
manhattan = 1 - tools.sigmoid(math_distance.manhattan_distance(combined_old[:,1], combined_old[:,2]))

print(abs(manhattan - manhattan_tuna))

Can we recover similarity function from scores and input vectors alone, which training strategies are best?

No cleaning manhattan distance

In [ ]:
sims = np.zeros(len(demo_matches))
for i in range(len(demo_matches)):

    q = demo_matches.iloc[i]['query']
    t = demo_matches.iloc[i]['target']

    q[:,1] /= sum(q[:,1])
    t[:,1] /= sum(t[:,1])
    combined = tools.match_peaks_in_spectra(q, t, ms2_da=0.05)
    sims[i] = 1- tools.sigmoid(math_distance.manhattan_distance(combined[:,1], combined[:,2]))

In [ ]:
reload(funcOb)
train_func = funcOb.func_ob(
    name = "test",
    sim_func = partial(tuna_sim, unnormed = 1),
    init_vals= [-2,0.6],
    params = ['dif_a','dif_b'],
    tol = 0,
    lambdas= 1,
    max_iter = 1000,
    epsilon = 1e-5
)

train = demo_matches[['query','target','precquery','prectarget']]
train['match'] = sims
train = train.sample(frac=1)

train_func.fit(train)

In [ ]:
train_func.init_vals

In [ ]:
train_func.converged

In [ ]:
train_func.trained_func

In [ ]:
tuna_sim

In [ ]:
from functools import partial

In [ ]:
partial(tuna_sim, unnormed = 1, dif_a = 1)(train.iloc[100]['query'], 
                                           train.iloc[100]['target'], 
                                           train.iloc[100]['precquery'], 
                                           train.iloc[100]['prectarget'])

In [ ]:
train_func.trained_func()(train.iloc[0]['query'], 
                                           train.iloc[100]['target'], 
                                           train.iloc[100]['precquery'], 
                                           train.iloc[100]['prectarget'])

In [ ]:
train.iloc[100]['match']

In [ ]:
epsilon = 1.4901161193847656e-08
#epsilon = 0.1
init = 0.01
insepct_loc = 0

upper = partial(tuna_sim, unnormed = 1, dif_a = init + epsilon)(train.iloc[insepct_loc]['query'], 
                                           train.iloc[insepct_loc]['target'], 
                                           train.iloc[insepct_loc]['precquery'], 
                                           train.iloc[insepct_loc]['prectarget'])

init = partial(tuna_sim, unnormed = 1, dif_a = init)(train.iloc[insepct_loc]['query'], 
                                           train.iloc[insepct_loc]['target'], 
                                           train.iloc[insepct_loc]['precquery'], 
                                           train.iloc[insepct_loc]['prectarget'])

actual = train.iloc[insepct_loc]['match']







In [ ]:
gradient = ((upper - actual)**2 - (init - actual)**2)/epsilon

In [ ]:
gradient